In [1]:
import numpy as np
import random
import networkx as nx
from matplotlib import pyplot as plt
import pandas as pd
import copy
import os
import sys
import networkx as nx
from scipy.sparse import lil_matrix, coo_matrix
from scipy.io import mmwrite

In [2]:
# moduleー
current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append( str(current_dir) + '/../' )

In [3]:
from setting_param import MakeSample_attribute_prediction_exist_InputDir as InputDir
from setting_param import MakeSample_attribute_prediction_exist_Age_OutputDir as OutputDir
from setting_param import L
from setting_param import attribute_dim

In [4]:
os.mkdir(OutputDir)
os.mkdir(OutputDir + "/input/")
os.mkdir(OutputDir + "/input/node_attribute/")
os.mkdir(OutputDir + "/input/adjacency")
os.mkdir(OutputDir + "/label/")
os.mkdir(OutputDir + "/mask/")

In [5]:
# READ EXIST_TABLE
EXIST_TABLE = np.load(InputDir + '/exist_table.npy')
EXIST_TABLE.shape

(3859, 68)

In [6]:
n_node = EXIST_TABLE.shape[0]

In [7]:
def ExistNodeList(ts):
    assert ts >= 0, "ts < 0 [referrence error]"
    return np.where(EXIST_TABLE[:, ts]==1)[0]

def GetAppearedNodes(ts):
    return set(ExistNodeList(ts)) - set(ExistNodeList(ts-1))

def GetObservedNodes(ts, L):
    U = set()
    for i in range(L):
        U |= set(ExistNodeList(ts-i))
    return U

def GetNodes(ts, L, node_type):
    if node_type=='all':
        node_set = set(ExistNodeList(ts))
    elif node_type=='stay':
        node_set = set(ExistNodeList(ts-1)) & set(ExistNodeList(ts))
    elif node_type=='lost':
        node_set = set(ExistNodeList(ts-1)) - set(ExistNodeList(ts))
    elif node_type=='return':
        node_set = GetAppearedNodes(ts) - (GetAppearedNodes(ts) - GetObservedNodes(ts-1, L))
    elif node_type=='new':
        node_set = GetAppearedNodes(ts) - GetObservedNodes(ts-1, L)
        node_set |= GetNodes(ts, L, 'return')
    return node_set

In [8]:
def Nx(ts):
    return  nx.from_numpy_matrix(np.load(InputDir + '/adjacency' + str(ts) + '.npy'))

def SubNxNew(ts, L):
    return nx.Graph(Nx(ts).edges(GetNodes(ts, L, 'new')))

def SubNxLost(ts, L):
    return nx.Graph(Nx(ts-1).edges(GetNodes(ts, L, 'lost')))

def GetEdges(ts, L, edge_type):
    G_1 = Nx(ts)
    if edge_type == "all":
        edge_set = G_1.edges
    elif edge_type == 'stay':
        G_0 = Nx(ts - 1)
        edge_set = G_0.edges & G_1.edges
    elif edge_type == "appeared":
        G_0 = Nx(ts - 1)
        edge_set = G_1.edges - G_0.edges - SubNxNew(ts, L).edges
    elif edge_type == "disappeared":
        G_0 = Nx(ts - 1)
        edge_set = G_0.edges - G_1.edges - SubNxLost(ts, L).edges
    return edge_set

In [9]:
def get_adjacency_matrix(ts, L, edge_type):
    G = nx.Graph(list(GetEdges(ts, L, edge_type)))
    A = np.array(nx.to_numpy_matrix(G, nodelist=[i for i in range(n_node)]))
    return A

def get_exist_matrix(ts):
    index = np.where(EXIST_TABLE[:, ts] == 1)[0]
    exist_row = np.zeros((n_node, n_node))
    exist_row[index] = 1
    exist_col = np.zeros((n_node, n_node))
    exist_col[:, index] = 1
    return exist_row * exist_col

def NodeAttribute(ts):
    return  np.load(InputDir + '/node_attribute' + str(ts) + '.npy')

In [10]:
def TsSplit(ts, L):
    ts_train = [(ts+l) for l in range(L)]
    ts_test = ts_train[-1]+1
    ts_all = ts_train.copy()
    ts_all.extend([ts_test])
    return ts_train, ts_test, ts_all

In [11]:
#Attribute Index
Tm_idx = [i for i in range(30)]
Pos_idx = [i for i in range(30, 33)]
Age_idx = [33]
PTS_idx = [34]

print(Tm_idx)
print(Pos_idx)
print(Age_idx)
print(PTS_idx)

for ts in range(L, EXIST_TABLE.shape[1]-L):
    ts_train, ts_test, ts_all = TsSplit(ts, L)
    node_attribute = np.zeros((n_node, attribute_dim * L))
    npy_adjacency_matrix = np.zeros((n_node, n_node * L))
    for idx, ts_ in enumerate(ts_train):
        node_attribute[:, attribute_dim*idx : attribute_dim*(idx+1)] = NodeAttribute(ts_)
        npy_adjacency_matrix[:, n_node*idx : n_node*(idx+1)] = get_adjacency_matrix(ts_, L, 'all')
    lil_adjacency_matrix = lil_matrix(npy_adjacency_matrix)
    lil_node_attribute = lil_matrix(node_attribute)
    mmwrite(OutputDir + "/input/node_attribute/" + str(ts), lil_node_attribute)
    mmwrite(OutputDir + "/input/adjacency/" + str(ts), lil_adjacency_matrix)
    
    label = NodeAttribute(ts_test)[:, Age_idx]
    mmwrite(OutputDir + "/label/" + str(ts), lil_matrix(label))
    
    mask = np.zeros((n_node, label.shape[1]))
    mask[sorted(GetNodes(ts_test, L, 'stay')), :] = 1
    mmwrite(OutputDir + "/mask/" + str(ts), lil_matrix(mask))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[30, 31, 32]
[33]
[34]
